In [1]:
import pandas as pd
import plotly.express as px
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#### 3.1
Visalization

In [2]:
df = pd.read_csv("advertising.csv")

fig = px.scatter_3d(df, x='TV', y='Radio', z='Sales', color='Newspaper')
fig.show()

In [3]:
fig = px.scatter_3d(df, x='TV', y='Newspaper', z='Sales', color='Radio')
fig.show()

2Data preprocessing and Building models

In [4]:
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
#
#
# def preprocess_data(
#         X_train: pd.DataFrame,
#         y_train: pd.Series,
#         standardize: bool = True,
# ) -> tuple[pd.DataFrame, pd.Series, StandardScaler, PolynomialFeatures]:
#     # 1. deleting outliers (like y=805 in our data1.csv)
#     mean_y = y_train.mean()
#     std_y = y_train.std()
#     mask = (y_train > mean_y - 3 * std_y) & (y_train < mean_y + 3 * std_y)
#     X_train = X_train[mask]
#     y_train = y_train[mask]
#
#     # 2. standardization
#     if standardize:
#         scaler = StandardScaler()
#         X_scaled = scaler.fit_transform(X_train)
#     else:
#         scaler = None
#         X_scaled = X_train
#
#     # 3. adding polynomial features to X
#     poly = PolynomialFeatures(degree=2, include_bias=False)
#     X_poly = poly.fit_transform(X_scaled)
#
#     return X_poly, y_train, scaler, poly
#
# def preprocess_test_data(X_test: pd.DataFrame, scaler: StandardScaler | None, poly: PolynomialFeatures):
#     if scaler is None:
#         X_scaled = X_test
#     else:
#         X_scaled = scaler.transform(X_test)
#     X_poly = poly.transform(X_scaled)
#     return X_poly

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = df[['TV', 'Radio', 'Newspaper']]
y = df['Sales']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


model_raw = LinearRegression(fit_intercept=False)
model_raw.fit(X_train, y_train)
y_raw_pred = model_raw.predict(X_test)


model_normalized = make_pipeline(
    MinMaxScaler(),
    LinearRegression(fit_intercept=False)
)
model_normalized.fit(X_train, y_train)
y_normalized_pred = model_normalized.predict(X_test)


model_standardized = make_pipeline(
    StandardScaler(),
    LinearRegression(fit_intercept=False)
)
model_standardized.fit(X_train, y_train)
y_standardized_pred = model_standardized.predict(X_test)


model_fit_intercept = LinearRegression(fit_intercept=True)
model_fit_intercept.fit(X_train, y_train)
y_fit_intercept_pred = model_fit_intercept.predict(X_test)


model_normalized_fit_intercept = make_pipeline(
    MinMaxScaler(),
    LinearRegression(fit_intercept=True)
)
model_normalized_fit_intercept.fit(X_train, y_train)
y_normalized_fit_intercept_pred = model_normalized_fit_intercept.predict(X_test)


model_standardized_fit_intercept = make_pipeline(
    StandardScaler(),
    LinearRegression(fit_intercept=True)
)
model_standardized_fit_intercept.fit(X_train, y_train)
y_standardized_fit_intercept_pred = model_standardized_fit_intercept.predict(X_test)


In [6]:
from sklearn.metrics import mean_squared_error, r2_score


def evaluate(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "R^2": r2_score(y_true, y_pred),
    }

results = pd.DataFrame([
    { "Model": "LinearRegression/raw", **evaluate(y_test, y_raw_pred) },
    { "Model": "LinearRegression/standardized", **evaluate(y_test, y_standardized_pred) },
    { "Model": "LinearRegression/normalized", **evaluate(y_test, y_normalized_pred) },
    { "Model": "LinearRegression/fit_intercept", **evaluate(y_test, y_fit_intercept_pred) },
    { "Model": "LinearRegression/standardized/fit_intercept", **evaluate(y_test, y_standardized_fit_intercept_pred) },
    { "Model": "LinearRegression/normalized/fit_intercept", **evaluate(y_test, y_normalized_fit_intercept_pred) },
])

results

,Model,MSE,R^2
0,LinearRegression/raw,6.415150,0.792397
1,LinearRegression/standardized,225.717208,-6.304504
2,LinearRegression/normalized,6.484352,0.790158
3,LinearRegression/fit_intercept,2.907757,0.905901
4,LinearRegression/standardized/fit_intercept,2.907757,0.905901
5,LinearRegression/normalized/fit_intercept,2.907757,0.905901
